<a href="https://colab.research.google.com/github/clarissa-souza/Airflow-Postgres-Docker/blob/main/apoio_dados_Sidra.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import urllib3
import pandas as pd
import json
from datetime import datetime

# Consumir API Sidra e gravar no diretório local

In [ ]:
http=urllib3.PoolManager()
urlPortalAPI='http://api.sidra.ibge.gov.br/values/t/1618/n1/all/v/all/p/all/c49/all/c48/all?formato=json'
response = http.request('GET',urlPortalAPI)

/usr/local/lib/python3.8/dist-packages/urllib3/connectionpool.py:842: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn((


In [ ]:
print(response.status)

200


In [ ]:
dataResponse=response.data.decode('utf-8')

In [ ]:
dataResponse

'[{"NC":"Nível Territorial (Código)","NN":"Nível Territorial","MC":"Unidade de Medida (Código)","MN":"Unidade de Medida","V":"Valor","D1C":"Brasil (Código)","D1N":"Brasil","D2C":"Variável (Código)","D2N":"Variável","D3C":"Mês (Código)","D3N":"Mês","D4C":"Ano da safra (Código)","D4N":"Ano da safra","D5C":"Produto das lavouras (Código)","D5N":"Produto das lavouras"},{"NC":"1","NN":"Brasil","MC":"1006","MN":"Hectares","V":"83856075","D1C":"1","D1N":"Brasil","D2C":"109","D2N":"Área plantada","D3C":"202211","D3N":"novembro 2022","D4C":"48584","D4N":"Safra 2021","D5C":"0","D5N":"Total"},{"NC":"1","NN":"Brasil","MC":"1006","MN":"Hectares","V":"69130271","D1C":"1","D1N":"Brasil","D2C":"109","D2N":"Área plantada","D3C":"202211","D3N":"novembro 2022","D4C":"48584","D4N":"Safra 2021","D5C":"39428","D5N":"1 Cereais, leguminosas e oleaginosas"},{"NC":"1","NN":"Brasil","MC":"1006","MN":"Hectares","V":"1376225","D1C":"1","D1N":"Brasil","D2C":"109","D2N":"Área plantada","D3C":"202211","D3N":"novembro 

In [ ]:
data_json=json.loads(dataResponse)

In [ ]:
df = pd.read_json(dataResponse)

In [ ]:
df.to_csv('teste/teste.csv',index=False)

In [ ]:
df

,unit,value,type,year_month,year_harvest,product
3,Hectares,1376225,Area plantada,202211,Safra 2021,Algodao herbaceo
4,Hectares,174421,Area plantada,202211,Safra 2021,Amendoim (1ª Safra)
5,Hectares,6928,Area plantada,202211,Safra 2021,Amendoim (2ª Safra)
6,Hectares,1680428,Area plantada,202211,Safra 2021,Arroz
7,Hectares,477008,Area plantada,202211,Safra 2021,Aveia
...,...,...,...,...,...,...
206,Toneladas,-,Producao,202211,Safra 2022,Juta
207,Toneladas,16722361,Producao,202211,Safra 2022,Laranja
208,Toneladas,18227561,Producao,202211,Safra 2022,Mandioca
209,Toneladas,3839590,Producao,202211,Safra 2022,Tomate


# Tratar o arquivo com pandas (Retirar todos os acentos)


In [ ]:
pd.set_option('display.max_rows', 210)

In [ ]:
df = pd.read_csv('/content/teste/teste.csv')

In [ ]:
df.drop(['NC','NN','MC','D1C','D1N','D2C','D3N','D4C','D5C'], axis=1, inplace=True)

In [ ]:
df.rename(columns={'MN':'unit','V':'value','D2N':'type','D3C':'year_month','D4N':'year_harvest','D5N':'product'}, inplace=True)

In [ ]:
df = df.drop(0)

In [ ]:
# Excluir as linhas repetidas 
filtro1 = df['product']!='1 Cereais, leguminosas e oleaginosas'
filtro2 = df['product']!='Total'
df = df.loc[filtro1 & filtro2]

In [ ]:
df['product'] = df['product'].str.replace('.','1')

<ipython-input-12-014af732ecc7>:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['product'] = df['product'].str.replace('.','1')


In [ ]:
# retirando os números da coluna product e substituir o caracter ª por a
df['product'] = df['product'].str.replace(r'\d+ \b', '', regex=True)

# Retirar todos os acentos 
df.replace('ç','c', regex=True, inplace=True)
df.replace(['ã','á'],'a', regex=True, inplace=True)
df.replace('é','e', regex=True, inplace=True)
df.replace('Á','A', regex=True, inplace=True)
df.replace('ú','u', regex=True, inplace=True)

In [ ]:
df['year_harvest'] = df['year_harvest'].str.replace('Safra','')
df['year_month']= pd.to_datetime(df['year_month'], format='%Y%m') 
df['year_harvest'] = df['year_harvest'].astype(int)
df['value'] = df['value'].str.replace('-','0')
df['value'] = df['value'].astype(int)
df['created_at']=pd.Timestamp.today() 

In [ ]:
df

,unit,value,type,year_month,year_harvest,product,created_at
3,Hectares,1376225,Area plantada,2022-11-01,2021,Algodao herbaceo,2022-12-14 23:23:54.691614
4,Hectares,174421,Area plantada,2022-11-01,2021,Amendoim (1ª Safra),2022-12-14 23:23:54.691614
5,Hectares,6928,Area plantada,2022-11-01,2021,Amendoim (2ª Safra),2022-12-14 23:23:54.691614
6,Hectares,1680428,Area plantada,2022-11-01,2021,Arroz,2022-12-14 23:23:54.691614
7,Hectares,477008,Area plantada,2022-11-01,2021,Aveia,2022-12-14 23:23:54.691614
...,...,...,...,...,...,...,...
206,Toneladas,0,Producao,2022-11-01,2022,Juta,2022-12-14 23:23:54.691614
207,Toneladas,16722361,Producao,2022-11-01,2022,Laranja,2022-12-14 23:23:54.691614
208,Toneladas,18227561,Producao,2022-11-01,2022,Mandioca,2022-12-14 23:23:54.691614
209,Toneladas,3839590,Producao,2022-11-01,2022,Tomate,2022-12-14 23:23:54.691614


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 198 entries, 3 to 210
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   unit          198 non-null    object        
 1   value         198 non-null    int64         
 2   type          198 non-null    object        
 3   year_month    198 non-null    datetime64[ns]
 4   year_harvest  198 non-null    int64         
 5   product       198 non-null    object        
 6   created_at    198 non-null    datetime64[ns]
dtypes: datetime64[ns](2), int64(2), object(3)
memory usage: 12.4+ KB


In [ ]:
df.to_csv('teste/testefinal.csv',index=True,index_label=id)